In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        if 'week5' in filename:
            weeks = os.path.join(dirname, filename)
        if 'players' in filename:
            players = os.path.join(dirname, filename)
        if 'pffScoutingData' in filename:
            pffScoutingData = os.path.join(dirname, filename)
        if 'plays' in filename:
            plays = os.path.join(dirname, filename)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-big-data-bowl-2023/players.csv
/kaggle/input/nfl-big-data-bowl-2023/week6.csv
/kaggle/input/nfl-big-data-bowl-2023/week2.csv
/kaggle/input/nfl-big-data-bowl-2023/pffScoutingData.csv
/kaggle/input/nfl-big-data-bowl-2023/week3.csv
/kaggle/input/nfl-big-data-bowl-2023/week8.csv
/kaggle/input/nfl-big-data-bowl-2023/games.csv
/kaggle/input/nfl-big-data-bowl-2023/week5.csv
/kaggle/input/nfl-big-data-bowl-2023/week7.csv
/kaggle/input/nfl-big-data-bowl-2023/week1.csv
/kaggle/input/nfl-big-data-bowl-2023/week4.csv
/kaggle/input/nfl-big-data-bowl-2023/plays.csv


In [2]:
!pip install pandarallel
from pandarallel import pandarallel as pl
pl.initialize(progress_bar=True, verbose=0)

# Loading the example data

In [3]:
pffScoutingData_df = pd.read_csv(pffScoutingData)
pffScoutingData_df.head()

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021090900,97,25511,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021090900,97,35481,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021090900,97,35634,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021090900,97,39985,Pass Route,HB-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0


In [4]:
plays = pd.read_csv(plays)
plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man
1,2021090900,137,(13:18) (Shotgun) D.Prescott pass deep left to...,1,1,10,DAL,TB,DAL,2,...,NaN,108.0,EMPTY,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 4 LB, 3 DB",TRADITIONAL,0,Cover-3,Zone
2,2021090900,187,(12:23) (Shotgun) D.Prescott pass short middle...,1,2,6,DAL,TB,DAL,34,...,NaN,76.0,SHOTGUN,"0 RB, 2 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,0,Cover-3,Zone
3,2021090900,282,(9:56) D.Prescott pass incomplete deep left to...,1,1,10,DAL,TB,TB,39,...,NaN,49.0,SINGLEBACK,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 3 LB, 4 DB",TRADITIONAL,1,Cover-3,Zone
4,2021090900,349,(9:46) (Shotgun) D.Prescott pass incomplete sh...,1,3,15,DAL,TB,TB,44,...,NaN,54.0,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,"3 DL, 4 LB, 4 DB",TRADITIONAL,0,Cover-3,Zone


In [5]:
player_df = pd.read_csv(players)
player_df.head()

,nflId,height,weight,birthDate,collegeName,officialPosition,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,28963,6-5,240,1982-03-02,"Miami, O.",QB,Ben Roethlisberger
2,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
3,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
4,30078,6-2,228,1982-11-24,Harvard,QB,Ryan Fitzpatrick


In [6]:
tracking_df =  pd.read_csv(weeks)
tracking_df.head()

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021100700,95,30869.0,1,2021-10-08T00:23:33.900,77.0,LA,right,22.93,33.04,0.02,0.02,0.00,102.26,129.51,None
1,2021100700,95,30869.0,2,2021-10-08T00:23:34.000,77.0,LA,right,22.93,33.04,0.01,0.02,0.00,101.64,128.68,None
2,2021100700,95,30869.0,3,2021-10-08T00:23:34.100,77.0,LA,right,22.92,33.05,0.01,0.01,0.01,100.73,127.26,None
3,2021100700,95,30869.0,4,2021-10-08T00:23:34.200,77.0,LA,right,22.92,33.05,0.01,0.01,0.01,100.73,130.90,None
4,2021100700,95,30869.0,5,2021-10-08T00:23:34.300,77.0,LA,right,22.91,33.05,0.01,0.01,0.01,99.55,134.16,None


In [7]:
gp_df = tracking_df[['gameId', 'playId']].drop_duplicates()
gp_df.reset_index(drop=True, inplace=True)
gp_df = gp_df[0:1]

In [8]:
gp_df

,gameId,playId
0,2021100700,95


# PreProcess Data

In [9]:
def Add_playerData_To_TrackingData(row ,playerDf):
    #apply to tracking data
    playerid = row['nflId']
    player = playerDf.loc[(playerDf['nflId'] == playerid)]
    #print(f'playerid :{playerid}')
    #print(f'player :{player}')
    try:
        nflId_displayName      = player.displayName.values[0]
        nflId_officialPosition = player.officialPosition.values[0]
        nflId_height           = player.height.values[0]
        nflId_weight           = player.weight.values[0]
    except:
        nflId_displayName=''
        nflId_officialPosition=''
        nflId_height=''
        nflId_weight=''
        
    row['nflId_displayName']      = nflId_displayName
    row['nflId_officialPosition'] = nflId_officialPosition
    row['nflId_height']           = nflId_height
    row['nflId_weight']           = nflId_weight
    return row
# Collect a record of the number of players in each position in every play.
# This will be used later to determine the maxiumum possible number of players
# for each position, which in turn determines how many features are needed
# to be collected (or zero-padded) at every position.
def get_position_counts(row,trackingDf):
    expr = (trackingDf['gameId'] == row['gameId']) & (trackingDf['playId'] == row['playId'])
    df = trackingDf.loc[expr]
    # 每frame參與play的位置數量應該相同
    # 在play中，所以只統計第一frame的位置數。
    vc = df.loc[df['frameId'] == 1, 'nflId_officialPosition'].value_counts()
    return pd.DataFrame(np.array([vc.values]), columns=vc.keys())

In [10]:
#把球員data塞進track data
player = tracking_df.parallel_apply(Add_playerData_To_TrackingData , axis=1,playerDf = player_df )

In [11]:
tracking_df = player

In [12]:
#統計每個position數量
pos_counts = gp_df.parallel_apply(get_position_counts, axis=1 , trackingDf = tracking_df)
pos_counts_max = pd.concat([df for df in pos_counts]).max().astype(int)
del pos_counts  # Mark temp object for deletion to free memory
pos_counts_max.head()

WR    3
T     2
DE    2
FS    2
CB    2
dtype: int64

In [13]:
pos_counts_max.keys()

Index(['WR', 'T', 'DE', 'FS', 'CB', 'G', 'QB', 'DT', 'MLB', 'TE', 'SS', 'C',
       'NT', 'RB', 'OLB', ''],
      dtype='object')

In [14]:
filter = (tracking_df['gameId'] == 2021100700) &(tracking_df['playId'] == 95)\
         &(tracking_df['frameId'] == 1) & (tracking_df['nflId_officialPosition'] == 'T')

tracking_df[filter]

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,nflId_displayName,nflId_officialPosition,nflId_height,nflId_weight
0,2021100700,95,30869.0,1,2021-10-08T00:23:33.900,77.0,LA,right,22.93,33.04,0.02,0.02,0.00,102.26,129.51,None,Andrew Whitworth,T,6-7,330
282,2021100700,95,42400.0,1,2021-10-08T00:23:33.900,79.0,LA,right,23.26,26.67,0.01,0.01,0.01,103.56,5.00,None,Rob Havenstein,T,6-8,328


## 依據play id 將各個位置的frameId資料整合 當作一個Sequence

In [15]:
def format_features(row,tracking_data_source_df=None, all_positions=None ,feature_cols=None):
   
    #print(f'row:{row}')
    filter = (tracking_data_source_df['gameId'] == row['gameId']) \
             & (tracking_data_source_df['playId'] == row['playId'])
    
    ''' 取出相對應的play資料'''
    play_df = tracking_data_source_df.loc[filter]
    '''取得frames的集合並排序'''
    frames = sorted(play_df['frameId'].unique())
    #print(f'frames:{frames}')
    
    frame_data = [] #容器用以裝入處理好的frame data
    
    '''  Iterate through every frame in the play'''
    for fid in frames:
         #print(f'fid:{fid}')
         features = np.array([])
         '''Iterate每個位置，不一定在play每個位置都會有''' 
         for pos in all_positions:
                if pos!='':
                    pos_data = play_df.loc[(play_df['frameId'] == fid) &
                                   (play_df['nflId_officialPosition'] == pos),
                                   feature_cols].values
                    
                    pos_data =np.nan_to_num(np.hstack(pos_data))
                    #print(f'pos:{pos}')
                    #print(f'Typr:{type(pos_data)}')
                    #print(f'pos_data:{pos_data}')
                    features = np.hstack((features, pos_data))    
         #print(f'features:{features}')
         print(f'features len:{len(features)}')
         frame_data.append(features) 
         #break 
          
    return frame_data


def get_pass_results(row , plays_df):
    
    return plays_df.loc[(plays_df['gameId'] == row['gameId']) &
                        (plays_df['playId'] == row['playId']),
                        'passResult'].values[0]

In [16]:
# These are the column names to use as feature data.
feature_label_df = gp_df.copy()
feature_label_df['labels'] = feature_label_df.parallel_apply(get_pass_results, axis=1 , plays_df=plays)
feature_cols = ['nflId','x', 'y', 's', 'a', 'dis', 'o', 'dir','nflId_weight']
all_positions = sorted(pos_counts_max.keys())
feature_label_df['features']= gp_df.parallel_apply(format_features, axis=1, args=[ tracking_df,all_positions,feature_cols])

features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198
features len:198


In [17]:
feature_label_df

,gameId,playId,labels,features
0,2021100700,95,C,"[[46180.0, 23.9, 29.93, 0.0, 0.0, 0.01, 60.14,..."
